In [ ]:
import os
print(f"当前目录: {os.getcwd()}")
if 'arch-gaussian' in os.listdir():
    JUPYTER_ROOT = os.getcwd()
    os.chdir('arch-gaussian')
    print(f"更改后的目录: {os.getcwd()}")

In [ ]:
import config
# 基础设置
config.scene_name = "TestCity7" # 输入文件夹名称
config.output_name = "TestCity7"  # 输出文件夹名称
config.sh_degree = 3  # 0~3， 改为0可以缩小文件大小，但是无法被现有的unity工具识别， 推荐使用默认3
config.epochs = 7000  # 训练轮次， 3000， 7000， 15000， 30000
config.loaded_iter = 3000 # 加载训练好的模型
# 高级设置
config.resolution = 1 # 是否压缩图像，1为原尺寸，-1为自动压缩到不大于1600像素， 支持2的次方的数字
config.densify_until_iter = 15000  # 致密化结束的轮次
config.densify_grad_threshold = 0.0002  # 致密化的阈值，越小增加的越快

# 配置结束，更新配置文件
config.update_colmap_args()
config.update_args()

# 打印修改过后的参数
config.print_updated_colmap_args()
config.print_updated_args()


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./src")
import os
import torch
import numpy as np
np.set_printoptions(suppress=True)
from config import args  # 正式导入args

## 创建scene info 与修复

In [ ]:
# 创建scene info
from manager.scene_manager import load_and_fix_scene

scene_info = load_and_fix_scene(args)

## 导入相机

In [ ]:
from manager.camera_manager import CameraManager
cm = CameraManager()
train_cameras, test_cameras = cm.create_cameras(args, scene_info)


## 创建gaussian对象

In [ ]:
# 创建gaussian对象

from manager.gaussian_manager import create_gaussian_from_ply, create_gaussian_from_scene_info
if args.loaded_iter:
    print("Creating gaussians from ply")
    gaussians = create_gaussian_from_ply(args.sh_degree, os.path.join(args.model_path,"point_cloud", "iteration_" + str(args.loaded_iter),"point_cloud.ply"))
else:
    print("Creating gaussians from scene info")
    gaussians = create_gaussian_from_scene_info(args.sh_degree, scene_info)

print(f"num points: {gaussians.get_xyz.shape[0]}")

## 使用封装好的Gaussian Manager管理gaussian

In [ ]:
from manager.gaussian_manager import GaussianManager
gm = GaussianManager(args, gaussians) # 从gaussians创建GaussianManager
cam = cm.pick_camera(0)
image = gm.render(cam, convert_to_pil=True)
image

## 三维高斯编辑

In [ ]:
from manager.display_manager import Box, Geometry, Drawer
bbox = (np.array([-0.5,0,-2]), np.array([0.5,0.5,-1.9]))

mask = gm.position_mask(*bbox)

box = Box(*bbox)
drawer = Drawer()
drawer.add_geometry(box)

with gm.virtual():
    gm.paint_by_mask(mask)
    image = gm.render(cam, convert_to_pil= True)
    image = drawer.draw(cam, image,overwrite=True)
image

In [ ]:
with gm.virtual():
    gm.set_color(0.5,mask)
    gm.add_color_noise(1, mask)
    image = gm.render(cam, convert_to_pil=True)
image

In [ ]:
with gm.virtual():
    gm.clear_features_rest(mask)
    gm.set_color(0.5, mask)
    gm.noise_position(mask, bbox)
    image = gm.render(cam, convert_to_pil=True)
image

In [ ]:
from utils.image_utils import get_pil_image, save_pil_image
import torchvision.transforms.functional as TF

def gt_socket(**kwargs):
    """
    ground truth socket
    作用为替换真实的gt image
    """
    iteration = kwargs['iteration']
    camera = kwargs['viewpoint_cam']

    gt_image = camera.original_image
    if iteration % 100 == 0:
        pil_gt_image = get_pil_image(gt_image)
        save_pil_image(pil_gt_image, os.path.join(args.model_path, "snap_shots", f"{iteration:05d}_gt_org.jpg"))
    new_gt_image = TF.adjust_hue(gt_image, 0.4)
    return new_gt_image

def loss_socket(**kwargs):
    """
    对真实计算出的loss进行更改
    """
    global mask
    iteration = kwargs['iteration']
    if mask is None or gm.gaussians._xyz.shape[0] != mask.shape[0]:
        print(f"mask updated at iter {iteration}")
        mask = gm.position_mask(*bbox)
    gm.clear_grads(~mask)

snap_count = 0
first_iter_end = 100+args.loaded_iter if args.loaded_iter else 100
def post_socket(**kwargs):
    """
    完成每一轮训练后的后处理内容
    """
    args = kwargs['args']
    iteration = kwargs['iteration']
    image = kwargs['image']
    gt_image = kwargs['gt_image']
    gaussians = kwargs['gaussians']

    global snap_count

    if (iteration < first_iter_end and iteration % 10 == 0) or (iteration % 100 == 0):
        pil_image = gm.render(cm.pick_camera(snap_count%len(cm.train_cameras[1.0])), convert_to_pil=True)
        snap_count += 1
        # pil_image = get_pil_image(image)
        save_path = os.path.join(args.model_path, "snap_shots", f"{snap_count:05d}.jpg")
        save_pil_image(pil_image, save_path)

        #pil_gt_iamge = get_pil_image(gt_image)
        #gt_save_path = os.path.join(args.model_path, "snap_shots", f"{iteration:05d}_gt.jpg")
        #save_pil_image(pil_gt_iamge, gt_save_path)

    if iteration % 1000 == 0:
        print(gaussians.get_xyz.shape)

In [ ]:
from manager.train_manager import train, init_output_folder
with gm.virtual():
    init_output_folder(args, scene_info)

    gm.clear_features_rest(mask)
    gm.set_color(0.5, mask)
    gm.noise_position(mask, bbox)

    train(args,scene_info, gm.gaussians, cm.train_cameras, gt_socket=gt_socket,loss_socket=loss_socket, post_socket=post_socket)
    image = gm.render(cam, convert_to_pil=True)
image

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append('./src')
import time
import uuid
from tqdm.auto import tqdm
from random import randint
from argparse import ArgumentParser, Namespace
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from scene import Scene, GaussianModel
from scene.dataset_readers import sceneLoadTypeCallbacks
from scene.cameras import MiniCam
from gaussian_renderer import render, network_gui

from utils.loss_utils import l1_loss, ssim
from utils.general_utils import safe_state
from utils.image_utils import psnr
from utils.camera_utils import loadCam, OrbitCamera
from utils.arg_utils import parse_args
import numpy as np
np.set_printoptions(suppress=True)
import dearpygui.dearpygui as dpg
import open3d as o3d

In [ ]:
import config
config.scene_name = "TestCity6"
config.epochs = 15000
config.update_args()
config.update_colmap_args()
from config import args, epochs
print(args)

In [ ]:
dataset, opt, pipe = parse_args(args)
ply_path = os.path.join(dataset.model_path,
                        "point_cloud",
                        "iteration_" + str(epochs),
                        "point_cloud.ply")
print(f"ply file path: {ply_path}")


In [ ]:
gaussians = GaussianModel(dataset.sh_degree)
gaussians.load_ply(ply_path)


In [ ]:
print(gaussians.get_xyz.shape)
print(gaussians._features_dc.shape)
print(gaussians._features_rest.shape)
print(gaussians._rotation.shape)
print(gaussians._scaling.shape)
print(gaussians.denom.shape)


In [ ]:
def get_gaussians_size(gaussians):
    _xyz_size = gaussians._xyz.element_size() * gaussians._xyz.nelement() / 1024 / 1024
    _features_dc_size = gaussians._features_dc.element_size() * gaussians._features_dc.nelement() / 1024 / 1024
    _features_rest_size = gaussians._features_rest.element_size() * gaussians._features_rest.nelement() / 1024 / 1024
    _scaling_size = gaussians._scaling.element_size() * gaussians._scaling.nelement() / 1024 / 1024
    _rotation_size = gaussians._rotation.element_size() * gaussians._rotation.nelement() / 1024 / 1024
    _opacity_size = gaussians._opacity.element_size() * gaussians._opacity.nelement() / 1024 / 1024
    max_radii2D_size = gaussians.max_radii2D.element_size() * gaussians.max_radii2D.nelement() / 1024 / 1024
    xyz_gradient_accum_size = gaussians.xyz_gradient_accum.element_size() * gaussians.xyz_gradient_accum.nelement() / 1024 / 1024
    denom_size = gaussians.denom.element_size() * gaussians.denom.nelement() / 1024 / 1024
    total_size = _xyz_size + _features_dc_size + _features_rest_size + _scaling_size + _rotation_size + _opacity_size + max_radii2D_size + xyz_gradient_accum_size + denom_size
    return total_size
print(f"gaussians内存占用估算： {get_gaussians_size(gaussians):.2f}MB")

In [ ]:
bg_color = [1, 1, 1] if dataset.white_background else [0, 0, 0]
background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

In [ ]:
# # creating cam
# scene_info = sceneLoadTypeCallbacks["Colmap"](args.source_path, args.images, args.eval)
# cam_info = scene_info.train_cameras[0]

In [ ]:
# cam = loadCam(args=args, id=0, cam_info=cam_info, resolution_scale=1)

In [ ]:
import json
# 打开JSON文件
with open(os.path.join(args.model_path, "cameras.json")) as f:
    # 使用json.load()方法加载JSON数据
    camera_info_list = json.load(f)

print(camera_info_list[0].keys())


In [ ]:
from scene.cameras import MiniCam

cam = camera_info_list[0]
mini_cam = MiniCam(np.array(cam['position']), np.array(cam['rotation']), cam['width'], cam['height'], cam['fx'], cam['fy'])
print(cam['width'])
print(cam['height'])
print(cam['fx'])
print(cam['fy'])
print(mini_cam.image_width)
print(mini_cam.image_height)
print(mini_cam.FoVx)
print(mini_cam.FoVy)


In [ ]:
print(mini_cam.get_transform())

In [ ]:

fov_degrees = 2 * math.degrees(math.atan(math.tan(fov_radians / 2) * (image_width / image_height)))

In [ ]:
mini_cam.set_transform(np.array([1,5,-1]), np.array([110,-10,20]))
print(mini_cam.get_transform())

In [ ]:
from PIL import Image
def render_img(_mini_cam, _gaussian_obj, _pipe, _background, scale = 1.0):
    with torch.no_grad():

        render_pkg = render(_mini_cam, _gaussian_obj, _pipe, _background)

        buffer_image = render_pkg["render"]
        # print(buffer_image.shape)
        # HH = int(_mini_cam.image_height * scale)
        # WW = int(_mini_cam.image_width * scale)
        # buffer_image = F.interpolate(
        #     buffer_image.unsqueeze(0),
        #     size=(HH, WW),
        #     mode="bilinear",
        #     align_corners=False,
        # ).squeeze(0)
        #
        # buffer_image = (
        #     buffer_image.permute(1, 2, 0)
        #     .contiguous()
        #     .clamp(0, 1)
        #     .contiguous()
        #     .detach()
        #     .cpu()
        #     .numpy()
        # )

        buffer_image = buffer_image.permute(1, 2, 0).clamp(0, 1).detach().cpu().numpy()
        buffer_image *=255
        buffer_image = buffer_image.astype(np.uint8)
        print(buffer_image.shape)
        print(buffer_image.dtype)
        return Image.fromarray(buffer_image.astype(np.uint8))

img = render_img(mini_cam, gaussians, pipe, background, scale=0.5)
img


In [ ]:
%load_ext autoreload
%autoreload 2
# 使用 %autoreload 2 会在模块被修改后自动重新加载。
from utils.transpose_utils import TransposeHelper, MatrixFormat

In [ ]:
import copy
modified_gaussians = copy.deepcopy(gaussians)


In [ ]:
# 将numpy数组转换为Open3D的PointCloud对象
point_cloud = o3d.geometry.PointCloud()

_xyz = modified_gaussians.get_xyz.detach().cpu().numpy()
print("xyz最小值:", np.min(_xyz))
print("xyz最大值:", np.max(_xyz))
# 使用reshape函数改变数组形状
_features_dc = modified_gaussians._features_dc.detach().cpu().numpy()
_features_dc = _features_dc.reshape((_features_dc.shape[0], _features_dc.shape[2]))
print("_features_dc最小值:", np.min(_features_dc))
print("_features_dc最大值:", np.max(_features_dc))
point_cloud.points = o3d.utility.Vector3dVector(_xyz)
# 设置点的颜色
point_cloud.colors = o3d.utility.Vector3dVector(_features_dc)

# 创建numpy数组表示三条线段的起点和终点
start_points = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
end_points = np.array([[10, 0, 0], [0, 30, 0], [0, 0, 50]])

# 创建LineSet对象
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(np.concatenate([start_points, end_points], axis=0))
print(np.concatenate([start_points, end_points], axis=0))
line_set.lines = o3d.utility.Vector2iVector(np.array([[0, 3], [1, 4], [2, 5]]))

# 创建三种颜色
colors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).astype(np.float32)

# 将颜色属性赋值给LineSet对象
line_set.colors = o3d.utility.Vector3dVector(np.concatenate([colors, colors], axis=0))
print(np.concatenate([colors, colors], axis=0))

# 将点云和线段添加到显示的几何图形列表中
# o3d.visualization.draw_geometries([point_cloud, line_set])

In [ ]:
transpose_helper = TransposeHelper(args, scene_info)
ground_up_vector = transpose_helper.get_ground_up_vector_auto(True)

In [ ]:
from scipy.spatial.transform import Rotation as R
def rotate_vec3(_pts:np.ndarray, _rotation:R)->np.ndarray:
    """
    input : [n, 3] (np.ndarray cpu float32)
            scipy.spatial.transform.Rotation
    output: [n, 3] (np.ndarray cpu float32)
    """
    with torch.no_grad():
        return _rotation.apply(_pts).astype(np.float32)


org_pts = modified_gaussians.get_xyz.detach().cpu().numpy()
rotation = transpose_helper.get_calibration_rotation()

rotated_pts = rotate_vec3(org_pts, rotation)
org_pts2 = rotate_vec3(rotated_pts, rotation.inv())

print("org_pts[0]:",org_pts[0], end="  ")
print(org_pts.dtype)
print('->')
print("rotated_pts[0]:",rotated_pts[0], end="  ")
print(rotated_pts.dtype)
print('->')
print("org_pts2[0]:",org_pts2[0], end="  ")
print(org_pts2.dtype)

In [ ]:
def rotate_quaternion(_quaternion:np.ndarray, _rotation:R)->np.ndarray:
    """
    input : [n, 4] (np.ndarray cpu float32)
            scipy.spatial.transform.Rotation
    output: [n, 4] (np.ndarray cpu float32)
    """
    org_r = R.from_quat(_quaternion)
    out_r = org_r * _rotation
    return out_r.as_quat().astype(np.float32)


org_quaternion = modified_gaussians.get_rotation.detach().cpu().numpy()
rotation = transpose_helper.get_calibration_rotation()
rotated_quaternion = rotate_quaternion(org_quaternion, rotation)
org_quaternion2 = rotate_quaternion(rotated_quaternion, rotation.inv())
print("org_quaternion[0]:",org_quaternion[0], end="  ")
print(org_quaternion.dtype)
print('->')
print("rotated_quaternion[0]:",rotated_quaternion[0], end="  ")
print(rotated_quaternion.dtype)
print('->')
print("org_quaternion2[0]:",org_quaternion2[0], end="  ")
print(org_quaternion2.dtype)


In [ ]:
"""apply to gaussian model"""
# apply pts
with torch.no_grad():
    modified_gaussians._xyz = torch.tensor(rotated_pts).cuda().requires_grad_(True)
    # modified_gaussians._rotation = torch.tensor(rotated_quaternion).cuda().requires_grad_(True)


In [ ]:
new_orbit_cam = copy.deepcopy(orbit_cam)
new_orbit_cam.orbit(1200,200)
new_orbit_cam.pan(0,0)
new_orbit_cam.radius = 1
print(new_orbit_cam.radius)
print(new_orbit_cam.center)
mini_cam.update(new_orbit_cam)

img = render_img(mini_cam, modified_gaussians, pipe, background, scale=1)
img

In [ ]:
# apply rotation
with torch.no_grad():
    # modified_gaussians._xyz = torch.tensor(rotated_pts).cuda().requires_grad_(True)
    modified_gaussians._rotation = torch.tensor(rotated_quaternion).cuda().requires_grad_(True)

In [ ]:
img = render_img(mini_cam, modified_gaussians, pipe, background, scale=1)
img

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import FileLink
from datetime import datetime
import uuid
new_orbit_cam = copy.deepcopy(orbit_cam)


orbit_x = 0
orbit_x_step = 100


font_path = "./fonts/arial.ttf"
font = ImageFont.truetype(font_path, 40)

# 或者指定 TrueType 字体文件（需要提供正确的字体文件路径）
# font_path = "path/to/your/font.ttf"
# font = ImageFont.truetype(font_path, font_size)
cache_path = f"cache/{uuid.uuid4()}"
image_paths = []
if not os.path.exists(cache_path):
    os.makedirs(cache_path)

for i in tqdm(range(1)):
    orbit_x += orbit_x_step
    new_orbit_cam.orbit(orbit_x_step,0)
    new_orbit_cam.pan(0,0)
    orbit_x += orbit_x_step
    mini_cam.update(new_orbit_cam)

    img = render_img(mini_cam, modified_gaussians, pipe, background, scale=1)

    # 在左上角添加文字
    draw = ImageDraw.Draw(img)
    text = f"orbit_x {orbit_x}"
    draw.text((5, 5), text, fill='red', font=font)
    image_path = os.path.join(cache_path, f"{i:05d}.jpg")
    image_paths.append(image_path)
    img.save(image_path)

print(image_paths[-1])
time.sleep(1)

path = os.path.join('arch-gaussian', image_paths[-1])
print(path)
download_link = FileLink(path)
download_link

In [ ]:
# 将numpy数组转换为Open3D的PointCloud对象
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(gaussians.get_xyz.detach().cpu().numpy())

# 创建numpy数组表示三条线段的起点和终点
start_points = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
end_points = np.array([[10, 0, 0], [0, 30, 0], [0, 0, 50]])

# 创建LineSet对象
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(np.concatenate([start_points, end_points], axis=0))
print(np.concatenate([start_points, end_points], axis=0))
line_set.lines = o3d.utility.Vector2iVector(np.array([[0, 3], [1, 4], [2, 5]]))

# 创建三种颜色
colors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).astype(np.float32)

# 将颜色属性赋值给LineSet对象
line_set.colors = o3d.utility.Vector3dVector(np.concatenate([colors, colors], axis=0))
print(np.concatenate([colors, colors], axis=0))

# 将点云和线段添加到显示的几何图形列表中
o3d.visualization.draw_geometries([point_cloud, line_set])

In [ ]:

with torch.no_grad():
    # 定义坐标范围
    min_coord = torch.tensor([0.0, 0.0, 0.0]).cuda()
    max_coord = torch.tensor([1, 1, 100]).cuda()

    mask = (modified_gaussians.get_xyz >= min_coord) & (modified_gaussians.get_xyz <= max_coord)
    selected_indices = torch.all(mask, dim=1)
    modified_gaussians._features_dc[selected_indices] = torch.tensor([-2.0,-2.0, -2.0]).cuda()
    print(modified_gaussians._features_rest.shape)
    modified_gaussians._features_rest[selected_indices] = torch.zeros([15,3]).cuda()
